KoELECTRA로 실험해봤으니 이번에는 XLM RoBERTa로 실험해본다. 

다국어가 지원되는 모델이니 KoELECTRA에서는 사용하지 못했던 영문키워드 피처도 한번 사용해본다. 

KoELECTRA를 사용해서 성공적이었던 Voting 기법을 사용한다. 

In [1]:
from transformers import * 
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import random 
import re 
import math 
from tqdm import tqdm 
import sklearn 
import torch 
import torch.nn as nn 
import torch.functional as f
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler 
from sklearn.model_selection import train_test_split 
import time 
import datetime 

PyTorch version 1.7.0+cu110 available.
TensorFlow version 2.5.0 available.


In [2]:
train = pd.read_csv("open/train.csv") 
test = pd.read_csv("open/test.csv") 

train.shape, test.shape, train['label'].nunique()

((174304, 13), (43576, 12), 46)

In [3]:
for column in train.columns:
    print(column)

index
제출년도
사업명
사업_부처명
계속과제여부
내역사업명
과제명
요약문_연구목표
요약문_연구내용
요약문_기대효과
요약문_한글키워드
요약문_영문키워드
label


In [4]:
num_classes = train['label'].nunique() # 46 

In [5]:
submission = pd.read_csv("open/sample_submission.csv") 

In [6]:
# clean unnecessary symbols
def clean_text(sent):
    sent_clean=re.sub("[^가-힣ㄱ-하-ㅣ]", " ", sent)
    return sent_clean

In [7]:
def split_text(s, overlap = 20, chunk_size = 50): 
    total = [] 
    partial = [] 
    if len(s.split()) // (chunk_size - overlap) > 0:  
        n = len(s.split()) // (chunk_size - overlap) 
    else: 
        n = 1 
    for w in range(n): 
        if w == 0: 
            partial = s.split()[:chunk_size] 
            total.append(" ".join(partial)) 
        else:  
            partial = s.split()[w*(chunk_size - overlap):w*(chunk_size - overlap) + chunk_size]
            total.append(" ".join(partial)) 
    return total

In [8]:
train['요약문_내용'] = train['요약문_연구목표'] + train['요약문_연구내용'] + train['요약문_기대효과'] 
test['요약문_내용'] = test['요약문_연구목표'] + test['요약문_연구내용'] + test['요약문_기대효과'] 

In [9]:
train['요약문_내용'].fillna('NAN',inplace=True) 
test['요약문_내용'].fillna('NAN',inplace=True)

In [10]:
train['사업명'].fillna('NAN',inplace=True) 
train['사업_부처명'].fillna('NAN',inplace=True) 
train['내역사업명'].fillna('NAN',inplace=True) 
train['과제명'].fillna('NAN',inplace=True) 
train['요약문_한글키워드'].fillna('NAN',inplace=True) 
train['요약문_영문키워드'].fillna('NAN',inplace=True)

In [11]:
contents = train['요약문_내용'].values 
feature1 = train['사업명'].values 
feature2 = train['사업_부처명'].values 
feature3 = train['내역사업명'].values 
feature4 = train['과제명'].values 
feature5 = train['요약문_한글키워드'].values 
feature6 = train['요약문_영문키워드'].values 
feature7 = train['label'].values 

train_data = {'사업명':[],'사업_부처명':[],'내역사업명':[],'과제명':[],'한글키워드':[],'영문키워드':[],'요약문':[],'label':[]} 

for i in tqdm(range(contents.shape[0]), position = 0, leave = True): 
    sample = str(contents[i]) 
    splitted_text = split_text(clean_text(sample)) 
    for t in splitted_text: 
        train_data['요약문'].append(t) 
        train_data['사업명'].append(clean_text(str(feature1[i])))
        train_data['사업_부처명'].append(clean_text(str(feature2[i]))) 
        train_data['내역사업명'].append(clean_text(str(feature3[i]))) 
        train_data['과제명'].append(clean_text(str(feature4[i])))  
        train_data['한글키워드'].append(feature5[i]) # no cleaning for this one
        train_data['영문키워드'].append(feature6[i]) # no cleaning for this one 
        train_data['label'].append(feature7[i])  

100%|██████████| 174304/174304 [01:26<00:00, 2016.55it/s]


In [12]:
train_data = pd.DataFrame(train_data)

In [13]:
train_data.head(2)

,사업명,사업_부처명,내역사업명,과제명,한글키워드,영문키워드,요약문,label
0,농업기초기반연구,농촌진흥청,농산물안전성연구,유전정보를 활용한 새로운 해충 분류군 동정기술 개발,"뉴클레오티드 염기서열, 분자마커, 종 동정, 침샘, 전사체","nucleotide sequence, molecular marker, species...",새로운 해충분류군의 동정기술 개발 및 유입확산 추적 가 외래 및 돌발해충의 발생조사...,24
1,농업기초기반연구,농촌진흥청,농산물안전성연구,유전정보를 활용한 새로운 해충 분류군 동정기술 개발,"뉴클레오티드 염기서열, 분자마커, 종 동정, 침샘, 전사체","nucleotide sequence, molecular marker, species...",의 돌발 및 외래해충 다 외래 및 돌발해충의 유전적 다양성 조사 시험곤충 나 의 해...,24


In [15]:
## Now we tokenize each data and make sure they all lie within the 512 tokenization range 
## if not check how many have token length greater than 512 

tokenizer = XLMRobertaTokenizer.from_pretrained("xlm-roberta-base") 

def roberta_tokenizer(sent, MAX_LEN):  
    encoded_dict = tokenizer.encode_plus(
        text = sent, 
        add_special_tokens = True, # add [CLS] and [SEP]
        pad_to_max_length = False, 
        return_attention_mask = True # constructing attention_masks 
    )  
    
    input_id = encoded_dict['input_ids'] 
    attention_mask = encoded_dict['attention_mask'] # differentiate padding from non padding 
    
    if len(input_id) > 512: # head + tail methodology 
        input_id = input_id[:129] + input_id[-383:] 
        attention_mask = attention_mask[:129] + attention_mask[-383:]  
        print("Long Text!! Using Head+Tail Truncation")
    elif len(input_id) <= 512: 
        input_id = input_id + [0]*(512 - len(input_id)) 
        attention_mask = attention_mask + [0]*(512 - len(attention_mask))
        
    return input_id, attention_mask 


In [16]:
train_data['data'] = train_data['사업명'] + " " + train_data['사업_부처명'] + " " + train_data['내역사업명'] + " " + train_data['과제명'] + " " + train_data['한글키워드'] + " " + train_data['영문키워드'] + " " + train_data['요약문'] 

train_data.head(2)


,사업명,사업_부처명,내역사업명,과제명,한글키워드,영문키워드,요약문,label,data
0,농업기초기반연구,농촌진흥청,농산물안전성연구,유전정보를 활용한 새로운 해충 분류군 동정기술 개발,"뉴클레오티드 염기서열, 분자마커, 종 동정, 침샘, 전사체","nucleotide sequence, molecular marker, species...",새로운 해충분류군의 동정기술 개발 및 유입확산 추적 가 외래 및 돌발해충의 발생조사...,24,농업기초기반연구 농촌진흥청 농산물안전성연구 유전정보를 활용한 새로운 해충 분류군 동...
1,농업기초기반연구,농촌진흥청,농산물안전성연구,유전정보를 활용한 새로운 해충 분류군 동정기술 개발,"뉴클레오티드 염기서열, 분자마커, 종 동정, 침샘, 전사체","nucleotide sequence, molecular marker, species...",의 돌발 및 외래해충 다 외래 및 돌발해충의 유전적 다양성 조사 시험곤충 나 의 해...,24,농업기초기반연구 농촌진흥청 농산물안전성연구 유전정보를 활용한 새로운 해충 분류군 동...


In [17]:
train_text = train_data['data'].values 
train_labels = train_data['label'].values


In [18]:
BATCH_SIZE = 32
NUM_EPOCHS = 30
VALID_SPLIT = 0.1 
MAX_LEN = 512 # max token size for BERT, ELECTRA

In [19]:
N = train_data.shape[0] 

input_ids = np.zeros((N, MAX_LEN),dtype=int)
attention_masks = np.zeros((N, MAX_LEN),dtype=int)
labels = np.zeros((N),dtype=int)

for i in tqdm(range(N), position=0, leave=True): 
    try:
        cur_str = train_text[i]
        cur_label = train_labels[i]
        input_id, attention_mask = roberta_tokenizer(cur_str, MAX_LEN=MAX_LEN) 
        input_ids[i,] = input_id 
        attention_masks[i,] = attention_mask 
        labels[i] = cur_label 
    except Exception as e: 
        print(e)
        print(cur_str)
        pass


  5%|▍         | 79995/1638867 [01:03<20:30, 1267.11it/s]

Long Text!! Using Head+Tail Truncation


  8%|▊         | 122951/1638867 [01:37<20:37, 1225.13it/s]

Long Text!! Using Head+Tail Truncation


  8%|▊         | 138492/1638867 [01:49<20:50, 1199.64it/s]

...

 89%|████████▉ | 1454996/1638867 [19:53<02:34, 1189.49it/s]

Long Text!! Using Head+Tail Truncation
Long Text!! Using Head+Tail Truncation


 95%|█████████▍| 1554888/1638867 [21:16<01:09, 1200.78it/s]

Long Text!! Using Head+Tail Truncation


100%|██████████| 1638867/1638867 [22:27<00:00, 1216.32it/s]


In [20]:
input_ids = torch.tensor(input_ids, dtype=int)
attention_masks = torch.tensor(attention_masks, dtype=int)
labels = torch.tensor(labels, dtype=int)

In [21]:
train_inputs, val_inputs, train_labels, val_labels = train_test_split(input_ids, labels, random_state = 42, test_size = VALID_SPLIT) 

train_attention_mask, val_attention_mask, _, _ = train_test_split(attention_masks, labels, random_state = 42, test_size = VALID_SPLIT) 


train_inputs.shape, train_attention_mask.shape, train_labels.shape


(torch.Size([1474980, 512]), torch.Size([1474980, 512]), torch.Size([1474980]))

In [22]:
train_data = TensorDataset(train_inputs, train_attention_mask, train_labels) 
train_sampler = RandomSampler(train_data) 
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=BATCH_SIZE) 

validation_data = TensorDataset(val_inputs, val_attention_mask, val_labels) 
validation_sampler = SequentialSampler(validation_data) 
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=BATCH_SIZE)


# Define XLM RoBERTa Base Model

In [23]:
model = XLMRobertaForSequenceClassification.from_pretrained("xlm-roberta-base", num_labels=46) 
model.cuda()

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense

XLMRobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(250002, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (La

In [24]:
optimizer = AdamW(model.parameters(), lr=2e-5, eps=1e-8)

epochs = 20 

total_steps = len(train_dataloader) * epochs 

scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0,
                                            num_training_steps = total_steps)


In [25]:
def flat_accuracy(preds, labels): 
    pred_flat = np.argmax(preds, axis=1).flatten() 
    labels_flat = labels.flatten() 
    return np.sum(pred_flat == labels_flat) / len(labels_flat) 

def format_time(elapsed):
    # 반올림
    elapsed_rounded = int(round((elapsed)))
    # hh:mm:ss으로 형태 변경
    return str(datetime.timedelta(seconds=elapsed_rounded))


In [ ]:
# 재현을 위해 랜덤시드 고정
device = torch.device("cuda")
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# 그래디언트 초기화
model.zero_grad()

# 에폭만큼 반복
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # 시작 시간 설정
    t0 = time.time()

    # 로스 초기화
    total_loss = 0

    # 훈련모드로 변경
    model.train()
        
    # 데이터로더에서 배치만큼 반복하여 가져옴
    for step, batch in enumerate(train_dataloader):
        # 경과 정보 표시
        if step % 100 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))
            print('  current average loss = {}'.format(total_loss / step))

        # 배치를 GPU에 넣음
        batch = tuple(t.to(device) for t in batch)
        
        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_labels = batch

        # Forward 수행                
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask, 
                        labels=b_labels)
        
        # 로스 구함
        loss = outputs[0]

        # 총 로스 계산
        total_loss += loss.item()

        # Backward 수행으로 그래디언트 계산
        loss.backward()

        # 그래디언트 클리핑
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # 그래디언트를 통해 가중치 파라미터 업데이트
        optimizer.step()

        # 스케줄러로 학습률 감소
        scheduler.step()

        # 그래디언트 초기화
        model.zero_grad()

    # 평균 로스 계산
    avg_train_loss = total_loss / len(train_dataloader)            

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epoch took: {:}".format(format_time(time.time() - t0)))
        
    # ========================================
    #               Validation
    # ========================================

    print("")
    print("Running Validation...")

    #시작 시간 설정
    t0 = time.time()

    # 평가모드로 변경
    model.eval()

    # 변수 초기화
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0

    # 데이터로더에서 배치만큼 반복하여 가져옴
    for batch in validation_dataloader:
        # 배치를 GPU에 넣음
        batch = tuple(t.to(device) for t in batch)
        
        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_labels = batch
        
        # 그래디언트 계산 안함
        with torch.no_grad():     
            # Forward 수행
            outputs = model(b_input_ids, 
                            token_type_ids=None, 
                            attention_mask=b_input_mask, 
                            labels = b_labels)
    
        
        loss = outputs[0] 
        logits = outputs[1] 
        
        # 로스 구함 
        eval_loss += loss.item() 
        
        # CPU로 데이터 이동
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        
        # 출력 로짓과 라벨을 비교하여 정확도 계산
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        eval_accuracy += tmp_eval_accuracy
        nb_eval_steps += 1
    
    avg_val_loss = eval_loss / len(validation_dataloader)            
    print("  Average validation loss: {0:.2f}".format(avg_val_loss))
    print("  Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
    print("  Validation took: {:}".format(format_time(time.time() - t0)))
    
    torch.save(model.state_dict(), "XLM_ROBERTA_BASE_voting_" + str(epoch_i + 1)) 
print("")
print("Training complete!")



======== Epoch 1 / 20 ========
Training...
  Batch   100  of  46,094.    Elapsed: 0:01:38.
  current average loss = 1.4725397345423699
  Batch   200  of  46,094.    Elapsed: 0:03:15.
  current average loss = 1.2922560261189937
  Batch   300  of  46,094.    Elapsed: 0:04:53.
  current average loss = 1.1751570998628935
  Batch   400  of  46,094.    Elapsed: 0:06:31.
  current average loss = 1.104554558992386
  Batch   500  of  46,094.    Elapsed: 0:08:08.
  current average loss = 1.0658078015446664
  Batch   600  of  46,094.    Elapsed: 0:09:46.
  current average loss = 1.0256028666098913
  Batch   700  of  46,094.    Elapsed: 0:11:23.
  current average loss = 0.9891520497415747
  Batch   800  of  46,094.    Elapsed: 0:13:01.
  current average loss = 0.9557355850748718
  Batch   900  of  46,094.    Elapsed: 0:14:40.
  current average loss = 0.9250540375709534
  Batch 1,000  of  46,094.    Elapsed: 0:16:18.
  current average loss = 0.8972076822817325
  Batch 1,100  of  46,094.    Elapsed

  Batch 9,000  of  46,094.    Elapsed: 2:26:35.
  current average loss = 0.45334260779101815
  Batch 9,100  of  46,094.    Elapsed: 2:28:14.
  current average loss = 0.451599676379586
  Batch 9,200  of  46,094.    Elapsed: 2:29:52.
  current average loss = 0.44997460020523844
  Batch 9,300  of  46,094.    Elapsed: 2:31:30.
  current average loss = 0.44848745059562467
  Batch 9,400  of  46,094.    Elapsed: 2:33:09.
  current average loss = 0.44675359478676097
  Batch 9,500  of  46,094.    Elapsed: 2:34:48.
  current average loss = 0.4451978530089714
  Batch 9,600  of  46,094.    Elapsed: 2:36:25.
  current average loss = 0.44347061349176026
  Batch 9,700  of  46,094.    Elapsed: 2:38:03.
  current average loss = 0.4422005961656801
  Batch 9,800  of  46,094.    Elapsed: 2:39:41.
  current average loss = 0.4407958461427871
  Batch 9,900  of  46,094.    Elapsed: 2:41:18.
  current average loss = 0.4393929759749785
  Batch 10,000  of  46,094.    Elapsed: 2:42:56.
  current average loss = 0.

  Batch 17,800  of  46,094.    Elapsed: 4:50:10.
  current average loss = 0.35991900302493274
  Batch 17,900  of  46,094.    Elapsed: 4:51:47.
  current average loss = 0.3592255276499066
  Batch 18,000  of  46,094.    Elapsed: 4:53:24.
  current average loss = 0.358477194036176
  Batch 18,100  of  46,094.    Elapsed: 4:55:02.
  current average loss = 0.3576598111647361
  Batch 18,200  of  46,094.    Elapsed: 4:56:40.
  current average loss = 0.35693744849728337
  Batch 18,300  of  46,094.    Elapsed: 4:58:18.
  current average loss = 0.35626006842033603
  Batch 18,400  of  46,094.    Elapsed: 4:59:56.
  current average loss = 0.35546389482599056
  Batch 18,500  of  46,094.    Elapsed: 5:01:33.
  current average loss = 0.3546896827033394
  Batch 18,600  of  46,094.    Elapsed: 5:03:11.
  current average loss = 0.35405243236208533
  Batch 18,700  of  46,094.    Elapsed: 5:04:49.
  current average loss = 0.3533939650180416
  Batch 18,800  of  46,094.    Elapsed: 5:06:26.
  current average

  Batch 26,600  of  46,094.    Elapsed: 7:13:45.
  current average loss = 0.30743561034278133
  Batch 26,700  of  46,094.    Elapsed: 7:15:24.
  current average loss = 0.3069021446720177
  Batch 26,800  of  46,094.    Elapsed: 7:17:02.
  current average loss = 0.3064694964320489
  Batch 26,900  of  46,094.    Elapsed: 7:18:39.
  current average loss = 0.3060298347360278
  Batch 27,000  of  46,094.    Elapsed: 7:20:17.
  current average loss = 0.30552462673152764
  Batch 27,100  of  46,094.    Elapsed: 7:21:54.
  current average loss = 0.30499384410232583
  Batch 27,200  of  46,094.    Elapsed: 7:23:32.
  current average loss = 0.3045381404138992
  Batch 27,300  of  46,094.    Elapsed: 7:25:10.
  current average loss = 0.3040049607987642
  Batch 27,400  of  46,094.    Elapsed: 7:26:47.
  current average loss = 0.3035764557403431
  Batch 27,500  of  46,094.    Elapsed: 7:28:24.
  current average loss = 0.30303383107362314
  Batch 27,600  of  46,094.    Elapsed: 7:30:02.
  current average

  Batch 35,400  of  46,094.    Elapsed: 9:37:08.
  current average loss = 0.2704354885783399
  Batch 35,500  of  46,094.    Elapsed: 9:38:46.
  current average loss = 0.27009217824094967
  Batch 35,600  of  46,094.    Elapsed: 9:40:24.
  current average loss = 0.26982849888075167
  Batch 35,700  of  46,094.    Elapsed: 9:42:03.
  current average loss = 0.2694109403347096
  Batch 35,800  of  46,094.    Elapsed: 9:43:40.
  current average loss = 0.26904903357185805
  Batch 35,900  of  46,094.    Elapsed: 9:45:17.
  current average loss = 0.26871466804627187
  Batch 36,000  of  46,094.    Elapsed: 9:46:55.
  current average loss = 0.26837857444282337
  Batch 36,100  of  46,094.    Elapsed: 9:48:33.
  current average loss = 0.26801246710400256
  Batch 36,200  of  46,094.    Elapsed: 9:50:10.
  current average loss = 0.267705332579809
  Batch 36,300  of  46,094.    Elapsed: 9:51:48.
  current average loss = 0.26732938424766206
  Batch 36,400  of  46,094.    Elapsed: 9:53:25.
  current avera

...

  Batch 12,500  of  46,094.    Elapsed: 3:23:27.
  current average loss = 0.020861855360281043
  Batch 12,600  of  46,094.    Elapsed: 3:25:05.
  current average loss = 0.020911618109699874
  Batch 12,700  of  46,094.    Elapsed: 3:26:42.
  current average loss = 0.020931062369229297
  Batch 12,800  of  46,094.    Elapsed: 3:28:19.
  current average loss = 0.02094291615876756
  Batch 12,900  of  46,094.    Elapsed: 3:29:56.
  current average loss = 0.02090777324235621
  Batch 13,000  of  46,094.    Elapsed: 3:31:34.
  current average loss = 0.020838380121592937
  Batch 13,100  of  46,094.    Elapsed: 3:33:12.
  current average loss = 0.020839070992771654
  Batch 13,200  of  46,094.    Elapsed: 3:34:50.
  current average loss = 0.020822262110169836
  Batch 13,300  of  46,094.    Elapsed: 3:36:28.
  current average loss = 0.020775277006475395
  Batch 13,400  of  46,094.    Elapsed: 3:38:06.
  current average loss = 0.020766722639212535
  Batch 13,500  of  46,094.    Elapsed: 3:39:43.
  c

  Batch 21,200  of  46,094.    Elapsed: 5:44:43.
  current average loss = 0.02042027645526405
  Batch 21,300  of  46,094.    Elapsed: 5:46:21.
  current average loss = 0.02043882541912921
  Batch 21,400  of  46,094.    Elapsed: 5:47:58.
  current average loss = 0.02045189785057875
  Batch 21,500  of  46,094.    Elapsed: 5:49:35.
  current average loss = 0.020513821600452755
  Batch 21,600  of  46,094.    Elapsed: 5:51:14.
  current average loss = 0.02050607669359427
  Batch 21,700  of  46,094.    Elapsed: 5:52:52.
  current average loss = 0.02050921221287496
  Batch 21,800  of  46,094.    Elapsed: 5:54:29.
  current average loss = 0.020463661930939243
  Batch 21,900  of  46,094.    Elapsed: 5:56:07.
  current average loss = 0.020476254171994054
  Batch 22,000  of  46,094.    Elapsed: 5:57:45.
  current average loss = 0.0204670229081058
  Batch 22,100  of  46,094.    Elapsed: 5:59:22.
  current average loss = 0.02051354601398677
  Batch 22,200  of  46,094.    Elapsed: 6:00:59.
  current

  Batch 29,900  of  46,094.    Elapsed: 8:06:01.
  current average loss = 0.020211122583779714
  Batch 30,000  of  46,094.    Elapsed: 8:07:40.
  current average loss = 0.020209104916204645
  Batch 30,100  of  46,094.    Elapsed: 8:09:18.
  current average loss = 0.020186510971907046
  Batch 30,200  of  46,094.    Elapsed: 8:10:56.
  current average loss = 0.020183001005618206
  Batch 30,300  of  46,094.    Elapsed: 8:12:33.
  current average loss = 0.020190245011573748
  Batch 30,400  of  46,094.    Elapsed: 8:14:11.
  current average loss = 0.020165620926707106
  Batch 30,500  of  46,094.    Elapsed: 8:15:49.
  current average loss = 0.020154359968235322
  Batch 30,600  of  46,094.    Elapsed: 8:17:27.
  current average loss = 0.020180755127425863
  Batch 30,700  of  46,094.    Elapsed: 8:19:04.
  current average loss = 0.020216334673313886
  Batch 30,800  of  46,094.    Elapsed: 8:20:42.
  current average loss = 0.02021665212013012
  Batch 30,900  of  46,094.    Elapsed: 8:22:19.
  

  Batch 38,600  of  46,094.    Elapsed: 10:27:53.
  current average loss = 0.02017691108243662
  Batch 38,700  of  46,094.    Elapsed: 10:29:31.
  current average loss = 0.02019281202275944
  Batch 38,800  of  46,094.    Elapsed: 10:31:10.
  current average loss = 0.020204095265929627
  Batch 38,900  of  46,094.    Elapsed: 10:32:47.
  current average loss = 0.02018323828793809
  Batch 39,000  of  46,094.    Elapsed: 10:34:26.
  current average loss = 0.02018282338715982
  Batch 39,100  of  46,094.    Elapsed: 10:36:03.
  current average loss = 0.020183221801365288
  Batch 39,200  of  46,094.    Elapsed: 10:37:41.
  current average loss = 0.020170035575826487
  Batch 39,300  of  46,094.    Elapsed: 10:39:18.
  current average loss = 0.020158700942495812
  Batch 39,400  of  46,094.    Elapsed: 10:40:56.
  current average loss = 0.020156374669296605
  Batch 39,500  of  46,094.    Elapsed: 10:42:33.
  current average loss = 0.020164099852719407
  Batch 39,600  of  46,094.    Elapsed: 10:4

  Batch 1,000  of  46,094.    Elapsed: 0:16:20.
  current average loss = 0.017255990364900754
  Batch 1,100  of  46,094.    Elapsed: 0:17:57.
  current average loss = 0.017626105013832785
  Batch 1,200  of  46,094.    Elapsed: 0:19:35.
  current average loss = 0.017544001187186496
  Batch 1,300  of  46,094.    Elapsed: 0:21:13.
  current average loss = 0.01710354141017258
  Batch 1,400  of  46,094.    Elapsed: 0:22:51.
  current average loss = 0.01676647365952249
  Batch 1,500  of  46,094.    Elapsed: 0:24:28.
  current average loss = 0.016412529344942715
  Batch 1,600  of  46,094.    Elapsed: 0:26:06.
  current average loss = 0.015692842239125752
  Batch 1,700  of  46,094.    Elapsed: 0:27:44.
  current average loss = 0.015739506101630576
  Batch 1,800  of  46,094.    Elapsed: 0:29:21.
  current average loss = 0.01599372071422345
  Batch 1,900  of  46,094.    Elapsed: 0:30:59.
  current average loss = 0.015859951057758927
  Batch 2,000  of  46,094.    Elapsed: 0:32:36.
  current avera

# Preprocess Test Data and make prediction

In [ ]:
# load best checkpoint model
checkpoint = torch.load('') 
test_model = XLMRobertaForSequenceClassification.from_pretrained("xlm-roberta-base", num_labels=46) 
test_model.load_state_dict(checkpoint) 
test_model.cuda()

In [ ]:
import torch.nn.functional as nnf

test['사업명'].fillna('NAN',inplace=True) 
test['사업_부처명'].fillna('NAN',inplace=True) 
test['내역사업명'].fillna('NAN',inplace=True) 
test['과제명'].fillna('NAN',inplace=True) 
test['요약문_한글키워드'].fillna('NAN',inplace=True)
test['요약문_영문키워드'].fillna('NAN',inplace=True)

In [ ]:
test_contents = test['요약문_내용'].values 
test_feature1 = test['사업명'].values 
test_feature2 = test['사업_부처명'].values 
test_feature3 = test['내역사업명'].values 
test_feature4 = test['과제명'].values 
test_feature5 = test['요약문_한글키워드'].values 
test_feature6 = test['요약문_영문키워드'].values 

predicted_classes = []

# change to eval mode 
test_model.eval() 

for i in tqdm(range(test_contents.shape[0]), position=0, leave=True):
    f1 = clean_text(str(test_feature1[i])) 
    f2 = clean_text(str(test_feature2[i])) 
    f3 = clean_text(str(test_feature3[i])) 
    f4 = clean_text(str(test_feature4[i])) 
    f5 = str(test_feature5[i])  
    f6 = str(test_feature6[i]) 
    splitted = split_text(str(test_contents[i])) 
    # make predictions for each splitted text 
    probabilities = [] 
    for text in splitted: 
        test_text = f1 + " " + f2 + " " + f3 + " " + f4 + " " + f5 + " " + f6 + " " + text 
        # tokenize test text 
        input_id, attention_mask = roberta_tokenizer(test_text, MAX_LEN=MAX_LEN) 
        input_id = torch.tensor(input_id)
        attention_mask = torch.tensor(attention_mask) 
        # reshape into (batch, MAX_LEN)
        input_id = torch.reshape(input_id, (-1,MAX_LEN)) 
        attention_mask = torch.reshape(attention_mask, (-1,MAX_LEN)) 
        # move tensor to cuda 
        input_id = input_id.to(device) 
        attention_mask = attention_mask.to(device) 
        
        with torch.no_grad(): 
            outputs = test_model(input_id, 
                            token_type_ids=None, 
                            attention_mask=attention_mask) 
        
        logits = outputs[0]
        
        # obtain softmax probabilities 
        prob = nnf.softmax(logits, dim=1).flatten()
        probabilities.append(prob)
    
    # soft voting 
    prob_sum = np.zeros(46) 
    for i in range(len(probabilities)): 
        for j in range(46): 
            prob_sum[j] += probabilities[i][j] 
            
    
    prob_sum /= len(probabilities)  
    
    predicted_class = np.argmax(prob_sum)
    
    predicted_classes.append(predicted_class)


In [ ]:
submission.iloc[:,1] = predicted_classes 

submission.head() 

In [ ]:
submission.to_csv("XLM_RoBERTa_BASE_voting.csv",index=False) 

# Ensembling Results from KoELECTRA and XLM RoBERTa 